In [1]:
import numpy as np
import pandas as pd 
import json
from collections import defaultdict
import copy
import pickle 

# Load APS corpus

In [2]:
class Paper(object):
    def __init__(self,
                 doi,
                 authors,
                 year,
                 full_abstract,
                 processed_abstract,
                 abstract_word_distr):
        
        self.doi = doi
        self.authors = authors
        self.year = year
        self.full_abstract = full_abstract
        self.processed_abstract = processed_abstract
        self.abstract_word_distr = abstract_word_distr

In [1]:
class APSCorpus(object):
    def __init__(self,
                 vocab,
                 papers_info_dict_list,
                 process = True):
        
        self.vocab = vocab
        self.id_to_word = defaultdict()
        self.word_to_id = defaultdict()
        self.papers = []
        self.doi_to_paperidx = defaultdict()
        
        if process:
            self.get_id_word_mapping()
            self.populate_papers(papers_info_dict_list)
            
    def get_id_word_mapping(self):
        # vocab_id to actual word mapping
        for idx,word in enumerate(self.vocab):
            self.id_to_word[idx] = word
            self.word_to_id[word] = idx
            
    def populate_papers(self,papers_info_dict_list):
        for paper_dict in papers_info_dict_list:
            self.doi_to_paperidx[paper_dict['doi']] = len(self.papers)
            self.papers.append(Paper(paper_dict['doi'],
                                     None,
                                     paper_dict['year'],
                                     paper_dict['full_abstract'],
                                     paper_dict['updated_abstract'],
                                     paper_dict['abstract_word_distr']
                                    ))

In [4]:
filehandler = open('data/apscorpus.obj', 'rb') 
corpus = pickle.load(filehandler)

In [10]:
id_ = 5
corpus.papers[id_].full_abstract

'An extension to transition state theory is presented that is capable of computing quantitatively the diffusivity of adsorbed molecules in confined systems at nonzero loading. This extension to traditional transition state theory yields a diffusivity in excellent agreement with that obtained by conventional molecular dynamics simulations. While molecular dynamics calculations are limited to relatively fast diffusing molecules or small rigid molecules, our approach extends the range of accessible time scales significantly beyond currently available methods. It is applicable in any system containing free energy barriers and for any type of guest molecule.'

In [11]:
corpus.papers[id_].processed_abstract

'extens transit state theori present capabl comput quantit diffus adsorb molecul confin system nonzero load extens tradit transit state theori yield diffus excel agreement obtain convent molecular dynam simul molecular dynam calcul limit relat fast diffus molecul small rigid molecul approach extend rang access time scale signific beyond current avail method applic system contain free energi barrier type guest molecul'

# Load FREX data

In [36]:
klist = [20,28,30,40,50,60]
frexlist = [0.25,0.5,0.75]

frex_concepts_dict = {key1:{key2:dict() for key2 in frexlist} for key1 in klist}

for k in klist:
    for frex in frexlist:
        filename = "data/frexconcepts_k"+str(k)+"_frex"+str(frex)+".json"
        f = open(filename)
        temp_dict = json.load(f)
        frex_concepts_dict[k][frex] = copy.deepcopy(temp_dict) # [20][0.75] holds the FREX concepts for k=20, FR=0.75

# Create FREX space

In [40]:
frex_space_dict = {key1:{key2:list() for key2 in frexlist} for key1 in klist}

for k in klist:
    for frex in frexlist:
        temp_frexspace = list(set([v for key,val in frex_concepts_dict[k][frex].items() for v in val]))
        frex_space_dict[k][frex] = copy.deepcopy(temp_frexspace)

# Export abstracts with only the FREX terms to disk

In [62]:
out_dict = {key1:{key2:list() for key2 in frexlist} for key1 in klist}
full_length_list = []
filtered_length_dict = {key1:{key2:list() for key2 in frexlist} for key1 in klist}

pid=0
for paper_id in range(len(corpus.papers)):
    current_abstract = corpus.papers[paper_id].processed_abstract
    temp_split_abs = current_abstract.split()
    full_length_list.append(len(temp_split_abs))
    
    #generate output for every (k,frex) combo
    for k in klist:
        for frex in frexlist:
            temp_refined_abs = [term for term in temp_split_abs if term in frex_space_dict[k][frex]]
            temp_out_abs = ' '.join(temp_refined_abs)
            out_dict[k][frex].append(temp_out_abs)
            filtered_length_dict[k][frex].append(len(temp_refined_abs))
            
    pid=pid+1
    if pid%1000==0:
        print(pid)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [63]:
for k in klist:
    for frex in frexlist:
        filename = "data/word2vec_inputs/abstracts_k"+str(k)+"_frex"+str(frex)+".txt"
        f = open(filename, "w")
        for abstr in out_dict[k][frex]:
            f.write(abstr + "\n")
        f.close()

In [65]:
for k in klist:
    for frex in frexlist:
        print("retained after frex filtering for ",k,frex,": ",
              np.mean(filtered_length_dict[k][frex])/np.mean(full_length_list)*100,"%")

retained after frex filtering for  20 0.25 :  20.793421048408813 %
retained after frex filtering for  20 0.5 :  51.10261391780746 %
retained after frex filtering for  20 0.75 :  86.38232059931083 %
retained after frex filtering for  28 0.25 :  27.424655080309858 %
retained after frex filtering for  28 0.5 :  61.66947869563481 %
retained after frex filtering for  28 0.75 :  89.89607989643875 %
retained after frex filtering for  30 0.25 :  36.11607402049927 %
retained after frex filtering for  30 0.5 :  67.08194834949994 %
retained after frex filtering for  30 0.75 :  90.36868748608694 %
retained after frex filtering for  40 0.25 :  52.144283198109896 %
retained after frex filtering for  40 0.5 :  84.50822266834757 %
retained after frex filtering for  40 0.75 :  91.23928348548827 %
retained after frex filtering for  50 0.25 :  64.16073116059724 %
retained after frex filtering for  50 0.5 :  89.83704041706918 %
retained after frex filtering for  50 0.75 :  91.5817801529203 %
retained afte

# Draft

In [33]:
f = open("data/frexconcepts_k40_frex0.5.json")
frexconcepts_k40_frex50 = json.load(f)

In [34]:
len(frexconcepts_k40_frex50.keys()), len(frexconcepts_k40_frex50['0'])

(40, 500)

In [35]:
print("Total concepts: "+str(len(frexconcepts_k40_frex50.keys())* len(frexconcepts_k40_frex50['0'])))

Total concepts: 20000


In [41]:
frexspace = list(set([v for key,val in frexconcepts_k40_frex50.items() for v in val]))

In [42]:
len(frexspace)

19041

In [43]:
frexspace[:10]

['hartmann',
 'tensor-product',
 'oxygen-cent',
 'cryocool',
 'least-squar',
 'τnl',
 'exposit',
 'epoxi',
 'system’',
 'cuauf']

In [66]:
len(corpus.papers)

249281

In [67]:
len(corpus.papers[0].processed_abstract.split())

39

In [68]:
split_abs = corpus.papers[id_].processed_abstract.split()
print(split_abs)

['extens', 'transit', 'state', 'theori', 'present', 'capabl', 'comput', 'quantit', 'diffus', 'adsorb', 'molecul', 'confin', 'system', 'nonzero', 'load', 'extens', 'tradit', 'transit', 'state', 'theori', 'yield', 'diffus', 'excel', 'agreement', 'obtain', 'convent', 'molecular', 'dynam', 'simul', 'molecular', 'dynam', 'calcul', 'limit', 'relat', 'fast', 'diffus', 'molecul', 'small', 'rigid', 'molecul', 'approach', 'extend', 'rang', 'access', 'time', 'scale', 'signific', 'beyond', 'current', 'avail', 'method', 'applic', 'system', 'contain', 'free', 'energi', 'barrier', 'type', 'guest', 'molecul']


In [69]:
refined_abs = [term for term in split_abs if term in frexspace]
refined_abs2 = ' '.join(refined_abs)
refined_abs2

'extens transit state theori present capabl comput quantit diffus adsorb molecul confin system load extens tradit transit state theori yield diffus excel agreement obtain convent molecular dynam simul molecular dynam calcul limit relat fast diffus molecul small rigid molecul approach extend rang access time scale signific beyond current avail method applic system contain free energi barrier type guest molecul'